# Data analysis following Galaxy epigenomics pipeline

On these notebooks, the output produced after running the two ChIP-Seq and the RNA-Seq workflows are processed to both continue the analysis and produce some figures and tables. Here are processed:
* ChIP-Seq
   * peak files
   * bam alignments
* RNA-Seq
   * gene counts
   * differential expression results

In this first notebook, ChIP-Seq data analysis is finished with the comparison of samples with MAnorm, followed by steps aimed at data visualization.

In [1]:
## setup working directory
jup_wd=~/work/jupyter-res
gal_wd=~/work/galaxy-res

[ ! -d ${jup_wd} ] && mkdir -p ${jup_wd}/figures
cd ${jup_wd}

## 1- Analysis of differentially marked regions

After the analysis of ChIP-Seq data from two samples, run on Galaxy worflows 1 and 2, the levels of ChIP-Seq reads on marked regions are compared. To achieve this, called peaks from both samples are compared to obtain a common set of peaks by MAnorm, which then calculates the normalized read density on these common regions to estimate the differences between samples. 

The data used on MAnorm were peaks called by epic2 with over 1 log2 fold change ChIP over INPUT signal, as a naive method to remove noise, and alignments with filtered low quality reads but keeping duplicates. To remove the epic2 log2FC filter, set `min_pk_fc=0`. Thus, the steps followed are:
* Filter epic2 peaks
* Run MAnorm

In [2]:
peak_caller=epic2
min_pk_fc=1

### Gather files
Galaxy exports several files to a results directory. For the next steps, bed and bam files are used.

In [3]:
## find input files on system
bed_01=($(ls ${gal_wd}/chipseq1/*bed | grep -i "${peak_caller}"))
bed_02=($(ls ${gal_wd}/chipseq2/*bed | grep -i "${peak_caller}"))

bam_1=($(ls ${gal_wd}/chipseq1/bam_files/*merged.bam* | grep -i chip))
inp_1=($(ls ${gal_wd}/chipseq1/bam_files/*merged.bam* | grep -i input))

bam_2=($(ls ${gal_wd}/chipseq2/bam_files/*merged.bam* | grep -i chip))
inp_2=($(ls ${gal_wd}/chipseq2/bam_files/*merged.bam* | grep -i input))

### Filter peak calling results
First, the remaining largest peaks after filtering by log2FC (from epic2, ChIP vs INPUT) are indicated for their visualization and evaluation on IGV.

In [4]:
# sample1
awk -v fc=$min_pk_fc '$7>fc{print $4,$3-$2+1,$1,$2,$3}' OFS='\t' \
   <(sort $bed_01) | sort -k2rn 2>/dev/null | head -5

island_37	21800	chr1	342200	363999
island_46	13800	chr1	474800	488599
island_25	8600	chr1	179800	188399
island_14	8200	chr1	116200	124399
island_49	7800	chr1	507600	515399


In [5]:
# sample2
awk -v fc=$min_pk_fc '$7>fc{print $4,$3-$2+1,$1,$2,$3}' OFS='\t' \
   <(sort $bed_02) | sort -k2rn 2>/dev/null | head -5

island_37	21000	chr1	343000	363999
island_47	11200	chr1	476800	487999
island_27	9400	chr1	179800	189199
island_91	8000	chr1	953200	961199
island_51	7400	chr1	507600	514999


Noisy peaks may also be removed when filtering for fold-change. Here no additional filtering is performed.

In [6]:
# prepare new names, remove galaxy history number, add FC
bed_1=$(echo "${bed_01}" | sed "s;/[0-9]\+_\(${peak_caller}.\+\).bed;/\1_FCgt${min_pk_fc}.bed;")
bed_2=$(echo "${bed_02}" | sed "s;/[0-9]\+_\(${peak_caller}.\+\).bed;/\1_FCgt${min_pk_fc}.bed;")

# print peaks that pass the log2FC threshold
awk -v fc=$min_pk_fc '$7>fc{print $0}' OFS='\t' \
   <(sort $bed_01) > ${bed_1}
awk -v fc=$min_pk_fc '$7>fc{print $0}' OFS='\t' \
   <(sort $bed_02) > ${bed_2}

In [7]:
# number of peaks before and after filtering
for bed in $bed_01 $bed_1 $bed_02 $bed_2
do
    wc -l $bed
done

90 /home/jovyan/work/galaxy-res/chipseq1/251_epic2_peaks1.bed
63 /home/jovyan/work/galaxy-res/chipseq1/epic2_peaks1_FCgt1.bed
93 /home/jovyan/work/galaxy-res/chipseq2/249_epic2_peaks2.bed
53 /home/jovyan/work/galaxy-res/chipseq2/epic2_peaks2_FCgt1.bed


### Differential binding: MAnorm 
The normalized read density on selected peaks was compared between the two samples. MAnorm compares the read density of peaks on a M-A plot to determine differentially marked regions. 

In [8]:
# setup
cd ${jup_wd}
sample_1=1
sample_2=2
manorm_dir=manorm-"$sample_1"vs"$sample_2"

# if manorm dir exists, delete to avoid issues
[ -d ${manorm_dir} ] && rm -r ${manorm_dir}

# run manorm
manorm \
--peak1 "$bed_1" \
--peak2 "$bed_2" \
--peak-format bed \
--read1 "$bam_1" \
--read2 "$bam_2" \
--read-format bam \
--name1 "$sample_1" \
--name2 "$sample_2" \
--paired-end \
-o "$manorm_dir" \
2> manorm.log

# add number of peaks passing each value to log
echo -e "\n# peaks\tM>0\tM>0.1\tM>0.25\tM>0.5\tM>1" >> manorm.log
awk -F '\t' 'NR>1{m_val=sqrt($5^2); if(m_val>0){a++;} if(m_val>.1){b++;} 
    if(m_val>.25){c++;} if(m_val>.5){d++;} if(m_val>1){e++;} }END{print "total",a,b,c,d,e}' \
    OFS='\t' "$manorm_dir"/*xls >> manorm.log
awk -F '\t' 'NR>1&&$5>0{m_val=$5; if(m_val>0){a++;} if(m_val>.1){b++;} 
    if(m_val>.25){c++;}if(m_val>.5){d++;} if(m_val>1){e++;} }END{print "M > 0",a,b,c,d,e}' \
    OFS='\t' "$manorm_dir"/*xls >> manorm.log
awk -F '\t' 'NR>1&&$5<0{m_val=-$5; if(m_val>0){a++;} if(m_val>.1){b++;} 
    if(m_val>.25){c++;} if(m_val>.5){d++;} if(m_val>1){e++;} }END{print "M < 0",a,b,c,d,e}' \
    OFS='\t' "$manorm_dir"/*xls >> manorm.log

# move log with the rest of files
mv manorm.log "$manorm_dir"

echo "$(( $(cat $manorm_dir/*xls | wc -l) - 1 )) combined peaks analyzed by MAnorm"
tail -4 $manorm_dir/manorm.log

69 combined peaks analyzed by MAnorm
# peaks	M>0	M>0.1	M>0.25	M>0.5	M>1
total	69	55	42	23	5
M > 0	42	36	29	16	3
M < 0	27	19	13	7	2


## 2 - Count ChIP-Seq reads on genomic bins 
For a sample-wise comparison of ChIP-Seq experiments as a Scatterplot, a previous step is to obtain read coverage on genomic bins. On this step, bam files of individual samples after removal of duplicated reads are used.


In [9]:
# bam files are renamed to remove the history number (this step can be skipped)
# A pattern is used to avoid targeting merged bam files
for f in ${gal_wd}/chipseq*/bam_files/*[^d].bam; do mv $f $(echo $f | sed 's;/[0-9]\{3\}_;/;'); done

In [10]:
# bam files are indexed with samtools.
cd ${gal_wd}/chipseq1/bam_files/
conda activate samtools

for f in *bam; do samtools index ${f} ${f}.bai; done
cd ${gal_wd}/chipseq2/bam_files/
for f in *bam; do samtools index ${f} ${f}.bai; done

conda deactivate

(samtools) (samtools) (samtools) (samtools) (samtools) (samtools) 

In [11]:
# run multiBamSummary 
cd ${jup_wd}
cpus=5
conda activate deeptools

multiBamSummary bins \
 --bamfiles $(ls ${gal_wd}/chipseq*/bam_files/*[^d].bam) \
 --binSize 1000 \
 --numberOfProcessors ${cpus} \
 -out ChIP_counts.npz \
 --outRawCounts ChIP_counts.tab \
 --scalingFactors ChIP_counts_sf.tab \
 2> ChIP_counts.log
 
conda deactivate

(deeptools) (deeptools) (deeptools) (deeptools) 

## 3 - Preparation for metagene plots and gene heatmaps
A metagene plot is a representation that helps visualize the distribution of ChIP-Seq reads across genes. For normalization, we used both ChIP and INPUT files. These results are further explored on the following R notebook.

A FIRST RUN WITHOUT DRAWING
is done to process the output and remove samples with no reads that were producing a weird band of no counts in our plots.

In [14]:
# setup
ngsplot_dir=${jup_wd}/figures/ngsplot
[ -d  $ngsplot_dir ] || mkdir -p $ngsplot_dir
cd  $ngsplot_dir
genes=~/work/lib/test_genome/genes.bed 
cpus=6

# s1
ngs.plot.r \
-P $cpus \
-G test \
-R bed \
-E "$genes" \
-C "$bam_1":"$inp_1" \
-O plot_genes_s1 \
-FI 1

# s2
ngs.plot.r \
-P $cpus \
-G test \
-R bed \
-E "$genes" \
-C "$bam_2":"$inp_2" \
-O plot_genes_s2 \
-FI 1


Configuring variables...Done
Loading R libraries.....Done
Analyze bam files and calculate coverageWarning messages:
1: In headerIndexBam(bam.list) :
  Aligner for: /home/jovyan/work/galaxy-res/chipseq1/bam_files/241_2-MarkDupes_ChIP_merged.bam cannot be determined. Style of 
standard SAM mapping score will be used. Would you mind submitting an issue 
report to us on Github? This will benefit people using the same aligner.
2: In headerIndexBam(bam.list) :
  Aligner for: /home/jovyan/work/galaxy-res/chipseq1/bam_files/242_2-MarkDupes_INPUT_merged.bam cannot be determined. Style of 
standard SAM mapping score will be used. Would you mind submitting an issue 
report to us on Github? This will benefit people using the same aligner.
Warning message:
'isNotPrimaryRead' is deprecated.
Use 'isSecondaryAlignment' instead.
See help("Deprecated") 
........Done
Saving results...Done
Wrapping results up...sh: 1: : Permission denied
Warning message:
In system2(zip, args, input = input) : error in run